In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import urllib
import time
from urllib.request import urlopen
import re
import math
import csv
import os,os.path

In [2]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
def url_encoding(url):
    url = urllib.parse.urlsplit(url)
    url = list(url)
    url[2] = urllib.parse.quote(url[2])
    url = urllib.parse.urlunsplit(url)
    return url

In [4]:
def convert(val):
    lookup = {'k': 1000, 'm': 1000000, 'b': 1000000000}
    unit = val[-1]
    number = float(val[:-1])
    if unit in lookup:
        return lookup[unit] * number
    return int(val)


In [5]:
def save2csv(row):
    #out_l = [['host','uptime','nfsserver','nfs status']] 
    #with open('test.csv', 'wb') as out_f:           # Python 2
    with open(filename, 'a', newline='',encoding="utf-8") as out_f: # Python 3
        w = csv.writer(out_f, delimiter='\t')        # override for tab delimiter
        w.writerows(row)  

In [6]:
def login(email,password):
    #PhantomJS_loc = r"C:\Users\Kriti chaudhary\Desktop\phantomjs.exe"
    #driver = webdriver.PhantomJS(executable_path=PhantomJS_loc)
    url = "https://www.quora.com/"
    driver.get(url)
    elem = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, ".regular_login [name=email]"))
)
    elem.send_keys(email)

    elem = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, ".regular_login [name=password]"))
)

    elem.send_keys(password)
    elem.send_keys(Keys.ENTER)    

In [7]:
def stats(username):
    get_url = 'https://www.quora.com/profile/'+str(username)
    get_url = url_encoding(get_url)
    url = urlopen(get_url)
    content = url.read()
    soup = BeautifulSoup(content,"lxml")
    ul = soup.find('ul')
    all_div = ul.find_all('div')
    answers = all_div[0].find('span',attrs={'class':"list_count"})
    try:
        answers = int(answers.text.replace(',', ''))
    except:
        answers = 0
    #print(answers.text)

    questions = all_div[2].find('span',attrs={'class':"list_count"})
    try:
        questions = int(questions.text.replace(',', ''))
    except:
        questions = 0
    #print(questions.text)

    posts = all_div[5].find('span',attrs={'class':"list_count"})
    try:
        posts = int(posts.text.replace(',', ''))
    except:
        posts = 0
    #print(posts.text)

    blogs = all_div[7].find('span',attrs={'class':"list_count"})
    try:
        blogs = int(blogs.text.replace(',', ''))
    except:
        blogs = 0
    #print(blogs.text)

    followers = all_div[9].find('span',attrs={'class':"list_count"})
    try:
        followers = int(followers.text.replace(',', ''))
    except:
        followers = 0
    #print(followers.text)

    following = all_div[11].find('span',attrs={'class':"list_count"})
    try:
        following = int(following.text.replace(',', ''))
    except:
        following = 0
    #print(following.text)

    topics = all_div[13].find('span',attrs={'class':"list_count"})
    try:
        topics = int(topics.text.replace(',', ''))
    except:
        topics = 0
    #print(topics.text)
    
    #print(answers, questions, posts, blogs, followers, following, topics)
    return [answers, questions, posts, blogs, followers, following, topics]

In [8]:
def get_topics(username):
    #---------------------- TOPICS ---------------------------------- #
    get_url = 'https://www.quora.com/profile/'+str(username)+'/topics'
    get_url = url_encoding(get_url)
    url = urlopen(get_url)
    content = url.read()
    soup = BeautifulSoup(content,"lxml")
    t = soup.findAll('a', 'topic_name')
    topics=[]
    for topic in t:
        topics.append(topic.text)
    if len(topics)>=10 :
        topics = topics[:10]
    #print(topics)
    return topics

In [9]:
def get_profile_detail(username):
    get_url = 'https://www.quora.com/profile/'+str(username)
    get_url = url_encoding(get_url)
    url = urlopen(get_url)
    content = url.read()
    time.sleep(2)
    soup = BeautifulSoup(content,"lxml")
    
    profile_name=None
    profile_cred=None
    work=None
    school=None
    answer=None
    location=None
    language=None
    top_writer=None
    published_writer=None

    #---------------------- PROFILE CREDENTIAL ---------------------------------- #
    pro = soup.find('div',attrs={'class':"ProfileNameAndSig"})
    profile_name = pro.find('span',attrs={'class':"user"}).text
    profile_cred = pro.find('span',attrs={'class':"UserCredential"})

    if (profile_cred):
        profile_cred = profile_cred.text
    #print(profile_name)
    #print(profile_cred)
    
    contents = soup.find('div',attrs={'class':"contents"})

    #---------------------- WORK CREDENTIAL ---------------------------------- #

    #work_credential = contents.select('div[class*="WorkCredentialListItem"]')[0]
    regex = re.compile('.*WorkCredentialListItem.*')
    try:
        work_credential = contents.find('div',attrs={'class':regex})
    except:
        work_credential = None
        
    #print()
    
    if (work_credential):    
        work_profile = work_credential.find('span',attrs={'class':'UserCredential'})
        if (work_profile):
            work = work_profile.text

        work_detail = work_credential.find('span',attrs={'class':'detail_text'})
        if (work_detail):
            work = work + ' ,' + work_detail.text

        #print(work)


    #---------------------- SCHOOL CREDENTIAL ---------------------------------- #

    regex = re.compile('.*SchoolCredentialListItem.*')
    try:
        school_credential = contents.find('div',attrs={'class':regex})
    except:
        school_credential = None
        
    #print()
    
    if (school_credential):
        school_profile = school_credential.find('span',attrs={'class':'UserCredential'})
        if (school_profile):
            school = school_profile.text

        school_detail = school_credential.find('span',attrs={'class':'detail_text'})
        if (school_detail):
            school = school + ' ,' + school_detail.text

        #print(school)
            
    
    #---------------------- LOCATION CREDENTIAL ---------------------------------- #

    regex = re.compile('.*LocationCredentialListItem.*')
    try:
        location_credential = contents.find('div',attrs={'class':regex})
    except:
        location_credential = None
    #print()
    if (location_credential): 
        location_profile = location_credential.find('span',attrs={'class':'UserCredential'})
        if (location_profile):
            location = location_profile.text

        location_detail = location_credential.find('span',attrs={'class':'detail_text'})

        #print(location)
       

    #---------------------- LANGUAGE CREDENTIAL ---------------------------------- #

    regex = re.compile('.*LanguageCredentialListItem.*')
    try:
        language_credential = contents.find('div',attrs={'class':regex})
    except:
        language_credential = None
        
    #print()
    
    if (language_credential): 
        language_profile = language_credential.find('span',attrs={'class':'UserCredential'})
        if (language_profile):
            language = language_profile.text

        language_detail = language_credential.find('span',attrs={'class':'detail_text'})
        if (language_detail):
            language = language + ' ,' + language_detail.text

        #print(language)
    
    
    #---------------------- ANSWER CREDENTIAL ---------------------------------- #

    regex = re.compile('.*AnswerViewsAboutListItem.*')
    try:
        answer_credential = contents.find('div',attrs={'class':regex})
    except:
        answer_credential = None

    #print()
    
    if (answer_credential): 
        answer_profile = answer_credential.find('span',attrs={'class':'main_text'})
        if (answer_profile):
            answer = answer_profile.text

        answer_detail = answer_credential.find('span',attrs={'class':'detail_text'})
        if (answer_detail):
            answer = answer + ' ,' + answer_detail.text

        #print(answer)
    
    
    #---------------------- TOP WRITER CREDENTIAL ---------------------------------- #

    regex = re.compile('.*TopWriterAboutListItem.*')
    try:
        top_writer_credential = contents.find('div',attrs={'class':regex})
    except:
        top_writer_credential = None
    
    #print()
    
    if (top_writer_credential):    
        top_writer_profile = top_writer_credential.find('span',attrs={'class':'main_text'})
        if (top_writer_profile):
            top_writer = top_writer_profile.text

        top_writer_detail = top_writer_credential.find('span',attrs={'class':'detail_text'})
        if (top_writer_detail):
            #top_writer = top_writer + ' ,' + top_writer_detail.text
            top_writer = top_writer_detail.text
            
            found = re.search("more", top_writer)
            if found:
                index = found.start() 
                top_writer = top_writer[index+4:]
            
    #print(top_writer)


    #---------------------- PUBLISHED WRITER CREDENTIAL ---------------------------------- #

    regex = re.compile('.*PublishedWriterAboutListItem.*')
    try:
        published_writer_credential = contents.find('div',attrs={'class':regex})
    except:
        published_writer_credential = None
    
    #print()
    
    if (published_writer_credential):   
        published_writer_profile = published_writer_credential.find('span',attrs={'class':'main_text'})
        #if (published_writer_profile):
        #    published_writer = published_writer_profile.text

        published_writer_detail = published_writer_credential.find('span',attrs={'class':'detail_text'})
        if (published_writer_detail):
            #published_writer = published_writer + ' ,' + published_writer_detail.text
            published_writer = published_writer_detail.text
            
            found = re.search("more", published_writer)
            if found:
                index = found.start() 
                published_writer = published_writer[index+4:]
            
    #print(published_writer)  
    #print(profile_name, profile_cred, work, school, answer, location, language, top_writer, published_writer)
    return [profile_name, profile_cred, work, school, answer, location, language, top_writer, published_writer]

In [10]:
def get_maxupvotes(username):
    get_url = 'https://www.quora.com/profile/'+str(username)+'/answers?sort=views'
    get_url = url_encoding(get_url)
    driver.get(get_url)
    soup = BeautifulSoup(driver.page_source,"lxml")
    answers = soup.find('div',attrs={'class':'list_header'})
    total_answers = 0
    if len(answers.text)>7:
        answers = answers.text.split(' ')[0]
        if len(str(answers))>=3:
            answers = answers.replace(',', '')
        #print(answers)
        total_answers = int(answers)

    if (60<total_answers<100):
        total_answers = total_answers * 0.6
    if (100<=total_answers):
        total_answers = 100
    max_upvotes = 0
    
        
    #print("total no. of answers: ", total_answers)
    if(total_answers>0):
        ans_list = soup.find_all('div',attrs={'class':'pagedlist_item'})
        while(len(ans_list)<total_answers):
            driver.execute_script("window.scrollTo(0,   document.body.scrollHeight);")
            soup = BeautifulSoup(driver.page_source,"lxml")
            ans_list = soup.find_all('div',attrs={'class':'pagedlist_item'})
            time.sleep(2)
        soup = BeautifulSoup(driver.page_source,"lxml")
        #print(soup)
        u = []
        #u = soup.find_all('span',attrs={'class':'ui_button_count_optimistic_count'})
        u = soup.find_all('div',attrs={'class':'action_bar_inner u-flex'})
        ans = []
        for k in range(0,len(u),2):  
            y = u[k].find('span',attrs={'class':'ui_button_label'})
            ans.append(y.text)
        ans = ans[1:]
        if(len(ans)==1):
            max_upvotes = int(convert('0'+ans[0].split(' ')[0]))   
        if (len(ans)>1):
            max_upvotes = max([int(convert('0'+(i.split(' ')[0]))) for i in ans])   
    #print("max upvotes:",max_upvotes)
    return max_upvotes
    

In [30]:
#if __name__ == '__main__':
import time
def main():
    global driver
    global filename
    global email
    global pwd
    global username
    login(email,pwd)
    time.sleep(10)
    get_url = 'https://www.quora.com/profile/' + str(username) + '/followers'
    driver.get(get_url)
    soup = BeautifulSoup(driver.page_source,"lxml")
    time.sleep(3)

    followers = soup.find('div',attrs={'class':'list_header'})
    followers = followers.text.split(' ')[0]

    total_followers = int(followers.replace(',', ''))
    #print("total no. of followers: ", total_followers)
    tf = soup.findAll('a', 'user')
    while(len(tf)<total_followers):
        driver.execute_script("window.scrollTo(0,   document.body.scrollHeight);")
        soup = BeautifulSoup(driver.page_source,"lxml")
        tf = soup.findAll('a', 'user')
        time.sleep(2)
    tf = tf[-total_followers:]

    file_exist = os.path.exists(filename)
    if (file_exist):
        os.remove(filename)
    
        
    columns = [['Username','Profile_Credential','Work_at','Education','Answer_viewed','Location','Language',\
                   'Top_writer','Published_writer','Answers','Questions','Posts','Blogs',\
                   'Followers','Following','Topics','Max_Upvotes','Knows_about']]
    save2csv(columns)

    user_name = username
    #print(user_name)
    pro_detail = get_profile_detail(user_name)
    statistics = stats(user_name)
    max_up = get_maxupvotes(user_name)
    topics = get_topics(user_name)
        
    pro_detail.extend(statistics)
    pro_detail.extend([max_up])
    pro_detail.append(topics)
        
    #print(pro_detail)
    save2csv([pro_detail])

    for user in tf:
        follower_name = str(user['href']).split("/profile/")[1]
        #print(follower_name)
        pro_detail = get_profile_detail(follower_name)
        statistics = stats(follower_name)
        max_up = get_maxupvotes(follower_name)
        topics = get_topics(follower_name)
        
        pro_detail.extend(statistics)
        pro_detail.extend([max_up])
        pro_detail.append(topics)
        
        #print(pro_detail)
        save2csv([pro_detail])
        
        time.sleep(1)



In [37]:
if __name__ == '__main__': 
    
    chromedriver_loc = r"C:\Users\Kriti chaudhary\Desktop\chromedriver_win32\chromedriver.exe" #enter chromedriver path
    PhantomJS_loc = r"C:\Users\Kriti chaudhary\Desktop\phantomjs.exe"
    
    #driver = webdriver.PhantomJS(executable_path=PhantomJS_loc)
    driver = webdriver.Chrome(executable_path=chromedriver_loc)
    
    filename = 'My_QData.csv' # Where to store data
    
    # ------------------ login credential ----------------
    email =  # enter email
    pwd =  #  enter password
    # ----------------------------------------------------
    
    username = 'Aditya-Lunawat' # enter username
    
    main()

IncompleteRead: IncompleteRead(94804 bytes read)

In [45]:
import pandas as pd
df = (pd.read_csv('My_QData.csv',sep='\t'))
print(df.columns)
df

Index(['Username', 'Profile_Credential', 'Work_at', 'Education',
       'Answer_viewed', 'Location', 'Language', 'Top_writer',
       'Published_writer', 'Answers', 'Questions', 'Posts', 'Blogs',
       'Followers', 'Following', 'Topics', 'Max_Upvotes', 'Knows_about'],
      dtype='object')


,Username,Profile_Credential,Work_at,Education,Answer_viewed,Location,Language,Top_writer,Published_writer,Answers,Questions,Posts,Blogs,Followers,Following,Topics,Max_Upvotes,Knows_about
0,Aditya Lunawat,NaN,NaN,Studied at International Institute of Informat...,NaN,"Lives in Hyderabad, India",NaN,NaN,NaN,0,0,0,0,192,224,86,0,"['Startups', 'Computer Programming', 'Sports',..."
1,Anoop Vyas,NaN,NaN,NaN,NaN,"Lives in Hyderabad, India",Knows Middle English,NaN,NaN,0,0,0,0,0,60,36,0,"['Facebook (product)', 'Visiting and Travel', ..."
2,Aakash Gore,Senior Sales Officer,Senior Sales Officer at HDB Financial Service...,Studied Master of Technology in Computer Scien...,NaN,"Lived in Indore, Madhya Pradesh, India",NaN,NaN,NaN,5,0,0,0,3,172,365,1,"['Mathematics', 'Indore, Madhya Pradesh, India..."
3,Ishita Joshi,NaN,NaN,NaN,NaN,"Lives in Pune, Maharashtra, India",NaN,NaN,NaN,0,0,0,0,2,74,39,0,"['Science', 'Machine Learning', 'Miley Cyrus (..."
4,Khushboo Sen,NaN,NaN,NaN,NaN,"Lives in Indore, Madhya Pradesh, India",NaN,NaN,NaN,0,0,0,0,22,560,12,0,"['Association Football', 'Romantica (band)', '..."
5,Bhavika Joshi,NaN,NaN,NaN,NaN,"Lives in Bengaluru, Karnataka, India",NaN,NaN,NaN,0,1,0,0,10,144,37,0,"['Music', 'Visiting and Travel', 'Movies', 'Sc..."
6,Kanika Rathore,Travel enthusiast. Learner.,"Works at Infosys Limited ,2014-present",Studied at Medi-Caps Institute of Science and ...,"325.6k answer views ,579 this month",NaN,Knows Hindi,NaN,NaN,15,0,0,1,868,226,264,11100,"['Important Life Lessons', 'Visiting and Trave..."
7,Saroj Lunawat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,3,24,17,0,"['Sports', 'Business', 'Cooking', 'Science', '..."
8,Rajat Sharma,NaN,NaN,NaN,NaN,"Lives in Pune, Maharashtra, India",NaN,NaN,NaN,1,0,0,0,14,133,69,1,"['Investing', 'Finance', 'Investment Strategie..."
9,Akanksha Pendharkar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,4,75,16,0,"['Facebook (product)', 'Jonas Brothers', 'Frie..."


In [43]:
df.loc[df['Max_Upvotes'].idxmax()]

Username                                                 Kanika Rathore
Profile_Credential                          Travel enthusiast. Learner.
Work_at                          Works at Infosys Limited ,2014-present
Education             Studied at Medi-Caps Institute of Science and ...
Answer_viewed                       325.6k answer views ,579 this month
Location                                                            NaN
Language                                                    Knows Hindi
Top_writer                                                          NaN
Published_writer                                                    NaN
Answers                                                              15
Questions                                                             0
Posts                                                                 0
Blogs                                                                 1
Followers                                                       

In [44]:
df.loc[df['Followers'].idxmax()]

Username                                                Krishna Chavali
Profile_Credential                      web, mobile and application dev
Work_at                                                             NaN
Education                      Studied at Vasavi College of Engineering
Answer_viewed                        23.9k answer views ,373 this month
Location                                      Lives in Hyderabad, India
Language                                                            NaN
Top_writer                                                          NaN
Published_writer                                                    NaN
Answers                                                              35
Questions                                                             7
Posts                                                                 0
Blogs                                                                 0
Followers                                                       